In [ ]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 573.3 kB/s eta 0:00:00


In [ ]:
from keras.datasets import imdb
from keras_preprocessing.sequence import pad_sequences
import tensorflow as tf

In [ ]:
# let's load the dataset and then split
# it into training and testing sets
features = 2000
len = 50
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=features)


# we are using pad sequences to a fixed length
X_train = pad_sequences(X_train, maxlen=len)
X_test = pad_sequences(X_test, maxlen=len)

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
print(X_train.shape)
print(X_test.shape)

(25000, 50)
(25000, 50)


In [ ]:
print(y_train.shape)

(25000,)


In [ ]:
y_train[1:10]

array([0, 0, 1, 0, 0, 1, 0, 1, 0])

In [ ]:
X_train[1]

array([   2,    5,    2,  656,  245,    2,    5,    4,    2,  131,  152,
        491,   18,    2,   32,    2, 1212,   14,    9,    6,  371,   78,
         22,  625,   64, 1382,    9,    8,  168,  145,   23,    4, 1690,
         15,   16,    4, 1355,    5,   28,    6,   52,  154,  462,   33,
         89,   78,  285,   16,  145,   95], dtype=int32)

In [ ]:
# Getting all the words from word_index dictionary
word_idx = imdb.get_word_index()

# Originally the index number of a value and not a key,
# hence converting the index as key and the words as values
word_idx = {i: word for word, i in word_idx.items()}

# again printing the review
print([word_idx[i] for i in X_train[1]])

['and', 'to', 'and', 'finds', 'tv', 'and', 'to', 'of', 'and', 'these', 'thing', 'wants', 'but', 'and', 'an', 'and', 'cult', 'as', 'it', 'is', 'video', 'do', 'you', 'david', 'see', 'scenery', 'it', 'in', 'few', 'those', 'are', 'of', 'ship', 'for', 'with', 'of', 'wild', 'to', 'one', 'is', 'very', 'work', 'dark', 'they', "don't", 'do', 'dvd', 'with', 'those', 'them']


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional, SimpleRNN, Dense

**Unidirectional RNN**

In [ ]:
# Set the values for the embedding size and
# number of hidden units in the RNN layer
embedding = 128
hidden = 64

# Create a Sequential model object
model1 = Sequential()
model1.add(Embedding(features, embedding,
                    input_length=len))
model1.add(SimpleRNN(hidden))
model1.add(Dense(1, activation='sigmoid'))
model1.compile('adam', 'binary_crossentropy',
              metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 128)           256000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                12352     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 268417 (1.02 MB)
Trainable params: 268417 (1.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
#set batch size and number of epochs you want
batch_size = 32
epochs = 5

model1.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))

Epoch 1/5
782/782 [==============================] - 44s 56ms/step - loss: 0.1325 - accuracy: 0.9510 - val_loss: 0.8017 - val_accuracy: 0.7646
Epoch 2/5
782/782 [==============================] - 41s 52ms/step - loss: 0.1019 - accuracy: 0.9618 - val_loss: 0.8783 - val_accuracy: 0.7547
Epoch 3/5
782/782 [==============================] - 40s 52ms/step - loss: 0.0787 - accuracy: 0.9706 - val_loss: 1.0735 - val_accuracy: 0.7478
Epoch 4/5
782/782 [==============================] - 40s 52ms/step - loss: 0.0778 - accuracy: 0.9714 - val_loss: 1.1607 - val_accuracy: 0.6940
Epoch 5/5
782/782 [==============================] - 41s 53ms/step - loss: 0.0609 - accuracy: 0.9792 - val_loss: 1.1808 - val_accuracy: 0.7277


In [ ]:
# redefine model to output right after the first hidden layer
from tensorflow.keras.models import Model
model = Model(inputs=model1.inputs, outputs=model1.layers[0].output)

In [ ]:
features = model.predict(X_test[1])
features

2/2 [==============================] - 0s 4ms/step


array([[-0.00106409,  0.00824866, -0.06177601, ...,  0.07349478,
        -0.00321336,  0.0601719 ],
       [ 0.06690077,  0.1407671 ,  0.11387948, ..., -0.07172091,
        -0.01006031,  0.06057109],
       [ 0.06584854,  0.05550298,  0.10783064, ..., -0.07560354,
        -0.16340974, -0.04044654],
       ...,
       [ 0.11216965, -0.02822897,  0.03434373, ...,  0.16575745,
        -0.04417638, -0.10910487],
       [-0.01357045, -0.06024994,  0.02101284, ...,  0.06257634,
         0.03904661, -0.04472475],
       [-0.03808314,  0.00602651, -0.03074027, ..., -0.00720564,
         0.03058059,  0.03611491]], dtype=float32)

In [ ]:
features.shape

(50, 128)

In [ ]:
modell = Model(inputs=model1.inputs, outputs=model1.layers[1].output)

from numpy import expand_dims
test = expand_dims(X_test[1], axis=0)
features1 = modell.predict(test)
features1.shape

1/1 [==============================] - 0s 120ms/step


(1, 64)

In [ ]:
features1

array([[ 0.32096925,  0.30421415, -0.51208115, -0.12544009,  0.35478204,
         0.77132887, -0.6182165 ,  0.5002717 ,  0.5565328 ,  0.644142  ,
        -0.53504133,  0.24685372,  0.3491055 ,  0.5667802 , -0.39225712,
        -0.7219206 ,  0.24904081,  0.7040414 ,  0.09419011, -0.00671036,
        -0.4555903 , -0.01029287, -0.6439581 , -0.38241813,  0.46459112,
        -0.07471412, -0.5883243 ,  0.10084566,  0.07496008,  0.41974232,
        -0.45979035, -0.3968719 ,  0.49346355,  0.03045777,  0.2893414 ,
         0.7763445 ,  0.5831679 , -0.14925617,  0.84397584, -0.09846085,
        -0.21466005,  0.16398564,  0.11852946,  0.8432917 , -0.59958535,
         0.38087532,  0.55744755,  0.6256532 ,  0.62337977, -0.5187323 ,
         0.3822637 ,  0.22667266,  0.58393705, -0.4100626 , -0.78901213,
         0.14078994, -0.60938674,  0.0809363 ,  0.20822577,  0.2514704 ,
        -0.17318065,  0.43036172,  0.56032586, -0.0779899 ]],
      dtype=float32)

In [ ]:
# Evaluate the Model

loss, accuracy = model1.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)

782/782 [==============================] - 5s 7ms/step - loss: 0.6882 - accuracy: 0.7387
Test accuracy: 0.7387199997901917


**Bidirectional RNN**

In [ ]:
# Set the values for the embedding size and
# number of hidden units in the RNN layer
embedding = 128
hidden = 64

# Create a Sequential model object
model2 = Sequential()
model2.add(Embedding(features, embedding,
                    input_length=len))
model2.add(Bidirectional(SimpleRNN(hidden)))
model2.add(Dense(1, activation='sigmoid'))
model2.compile('adam', 'binary_crossentropy',
              metrics=['accuracy'])

print(model2.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 50, 128)           256000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               24704     
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 280833 (1.07 MB)
Trainable params: 280833 (1.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
#set batch size and number of epochs you want
batch_size = 32
epochs = 5

model2.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))

Epoch 1/5
782/782 [==============================] - 39s 46ms/step - loss: 0.5585 - accuracy: 0.6973 - val_loss: 0.4772 - val_accuracy: 0.7798
Epoch 2/5
782/782 [==============================] - 38s 49ms/step - loss: 0.4206 - accuracy: 0.8095 - val_loss: 0.4870 - val_accuracy: 0.7636
Epoch 3/5
782/782 [==============================] - 31s 40ms/step - loss: 0.3385 - accuracy: 0.8557 - val_loss: 0.5110 - val_accuracy: 0.7754
Epoch 4/5
782/782 [==============================] - 36s 46ms/step - loss: 0.2432 - accuracy: 0.9016 - val_loss: 0.6084 - val_accuracy: 0.7694
Epoch 5/5
782/782 [==============================] - 31s 40ms/step - loss: 0.1542 - accuracy: 0.9413 - val_loss: 0.7787 - val_accuracy: 0.7372


In [ ]:
# Evaluate the Model

loss, accuracy = model2.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)

782/782 [==============================] - 6s 8ms/step - loss: 0.7787 - accuracy: 0.7372
Test accuracy: 0.7371600270271301


**LSTM**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load the IMDB dataset
max_features = 10000  # number of words to consider as features
maxlen = 500  # maximum length of the review
batch_size = 64

# Load data from IMDB, already preprocessed as integer sequences
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# Pad sequences to ensure they have the same length (maxlen)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

# Build the LSTM model
model = Sequential()

# Embedding layer to map word indices to word embeddings
model.add(Embedding(input_dim=max_features, output_dim=128))

# LSTM layer
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

# Fully connected layer to produce final prediction
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Build the model by providing a batch input shape (required to resolve the issue)
model.build(input_shape=(None, maxlen))  # explicitly build the model

# Display model summary to verify the total parameters
model.summary()

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=batch_size,
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
score, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)

print(f"Test score: {score}")
print(f"Test accuracy: {accuracy}")


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 500, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,411,713 (5.39 MB)

 Trainable params: 1,411,713 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 716s 2s/step - accuracy: 0.6836 - loss: 0.5712 - val_accuracy: 0.8407 - val_loss: 0.3713
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 753s 2s/step - accuracy: 0.8589 - loss: 0.3459 - val_accuracy: 0.7698 - val_loss: 0.4775
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 729s 2s/step - accuracy: 0.8724 - loss: 0.3125 - val_accuracy: 0.8447 - val_loss: 0.3783
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 706s 2s/step - accuracy: 0.9028 - loss: 0.2461 - val_accuracy: 0.8496 - val_loss: 0.3632
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 741s 2s/step - accuracy: 0.9182 - loss: 0.2161 - val_accuracy: 0.8523 - val_loss: 0.4272
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 741s 2s/step - accuracy: 0.9324 - loss: 0.1836 - val_accuracy: 0.8467 - val_loss: 0.3989
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 740s 2s/step - accuracy: 0.9505 - loss: 0.1397 - val_accuracy: 0.8420 - val_loss: 0.4375
391/391 ━━━━━━━━━━━━━━━━━━━━ 103s 264ms/step - accuracy: 0.8400 - loss: 0.4424
Test score: 0.4375

**BI-LSTM**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load a simple text dataset or use your own dataset
text_data = """This is an example sentence for word prediction.
               The machine learns to predict the next word in a sentence.
               BiLSTM can capture the dependencies between words.
               Word prediction is important in natural language processing."""

# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_data])
sequences = tokenizer.texts_to_sequences([text_data])[0]

# Prepare input and output sequences
sequence_length = 5  # Lookback for previous words
input_sequences = []
output_words = []

# Create input sequences (x) and output words (y)
for i in range(sequence_length, len(sequences)):
    input_sequences.append(sequences[i-sequence_length:i])  # last `sequence_length` words
    output_words.append(sequences[i])  # the word to predict

# Pad the input sequences to ensure they are all of the same length
X = np.array(input_sequences)
y = np.array(output_words)

# The output labels need to be one-hot encoded
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index)+1)

# Model parameters
embedding_dim = 100
hidden_units = 128

# Build the BiLSTM model for next word prediction
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=embedding_dim, input_length=sequence_length))
model.add(Bidirectional(LSTM(hidden_units)))
model.add(Dropout(0.2))
model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50, batch_size=32)

# Save the model
model.save('next_word_prediction_model.h5')


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.0000e+00 - loss: 3.2966
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.1034 - loss: 3.2884
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.1724 - loss: 3.2817
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.2414 - loss: 3.2741
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.2759 - loss: 3.2659
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.2759 - loss: 3.2591
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.2414 - loss: 3.2465
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.2414 - loss: 3.2400
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.2414 - loss: 3.2314
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.2759 - loss: 3.2185
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.2414 - loss: 3.2096
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.2069 - loss: 3.2007
Epoch 1

In [ ]:
# Function to predict the next word based on input sequence
def predict_next_word(model, tokenizer, text, sequence_length=5):
    # Tokenize the input text
    sequence = tokenizer.texts_to_sequences([text])[0]

    # Ensure that the input sequence is the correct length
    sequence = sequence[-sequence_length:]

    # Pad the sequence to make it the correct length for input
    sequence = pad_sequences([sequence], maxlen=sequence_length)

    # Predict the next word
    pred = model.predict(sequence, verbose=0)

    # Get the index of the predicted word
    predicted_index = np.argmax(pred)

    # Get the word corresponding to the predicted index
    predicted_word = tokenizer.index_word.get(predicted_index, '')

    return predicted_word

# Example: Predict the next word for a given sequence
input_text = "BiLSTM can capture"
predicted_word = predict_next_word(model, tokenizer, input_text)
print(f"Predicted next word: {predicted_word}")


Predicted next word: dependencies
